<a href="https://colab.research.google.com/github/10isha/KBC-prize-prediction-challenge/blob/main/KBC_Prize_Prediction_Kaggle_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing Training Data Set

In [ ]:
train_data = pd.read_csv("/kaggle/input/kbc-prize-prediction-challenge/train.csv")
train_data.head()

Importing Test Data Set

In [ ]:
test_data = pd.read_csv("/kaggle/input/kbc-prize-prediction-challenge/test.csv")
test_data.head()

In [ ]:
train_data.isnull().any()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.columns

In [ ]:
train_data = train_data.dropna()
train_data.isnull().sum()


Removing Punctuation from Text

In [ ]:
import string 
string.punctuation

def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    return punctuationfree

train_data['clean_msg'] = train_data['Question'].apply(lambda x : remove_punctuation(x))
test_data['clean_msg'] = test_data['Question'].apply(lambda x : remove_punctuation(x))
train_data.head()

Tokenization

In [ ]:
import nltk
def tokenization(text):
    tokens = nltk.word_tokenize(text)
    return tokens 
train_data['msg_tokenized']= train_data['clean_msg'].apply(lambda x : tokenization(x))
test_data['msg_tokenized']= test_data['clean_msg'].apply(lambda x : tokenization(x))
train_data.head()

Removal of Stop Words from Text

In [ ]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output = [i for i in text if i not in stopwords]
    return output
train_data['no_stopwords'] = train_data['msg_tokenized'].apply(lambda x: remove_stopwords(x))
test_data['no_stopwords'] = test_data['msg_tokenized'].apply(lambda x: remove_stopwords(x))
stopwords
train_data.head()

Lemmatization

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [lemm.lemmatize(word) for word in text]
    return lemm_text
train_data['FinalTxt'] = train_data['no_stopwords'].apply(lambda x : lemmatizer(x))
test_data['FinalTxt'] = test_data['no_stopwords'].apply(lambda x : lemmatizer(x))
train_data.head()

Applying TF/IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer()
X= tf_idf.fit_transform(train_data['FinalTxt'].apply(lambda x: ''.join(x)))
X = tf_idf.transform(train_data['FinalTxt'].apply(lambda x: ''.join(x)))
print("n_samples: %d, n_features: %d" % X.shape)

In [ ]:
X_test = tf_idf.transform(test_data['FinalTxt'].apply(lambda x : ''.join(x)))
print("n_samples: %d, n_features: %d" % X_test.shape)

In [ ]:
y= train_data['Prize']
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.50, random_state = 0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
Prize_detect_model = MultinomialNB().fit(X_train, y_train)

In [ ]:
X_pred=Prize_detect_model.predict(X_test)

In [ ]:
output = pd.DataFrame({'Id': test_data.Id,
                       'Prize':X_pred})
output.to_csv('submission.csv', index=False)

In [ ]:
print(output)